In [1]:
## script to generate Scispot registrations for ChESS 96-well plate
## lkp 2022/11/15

import pandas as pd
import requests
import json
import sys

sys.stdout.write("Imported required packages")

Imported required packages

In [7]:
# Fetch rows from the sample manager
# https://www.loom.com/share/32c01c2cb1294720b15596b0a43af5e9
def fetch_samples():
   session = requests.Session()
   url = 'https://api.scispot.io/tryingtofixcors/labsheets/list-rows'
   payload = {
       "apiKey": "3F4E0776-EFEF-4A5B-8479-01B7C2367090",
       "manager": "Sample Manager",
       "page": "1",
       "pageSize": "3"
   }
   ret = session.post(url, json=payload)
   return json.loads(ret.text)

test = fetch_samples()
test

# turn json into pandas df
rows = test["rows"]
column_labels = test['headers']
df = pd.DataFrame(data=rows, columns=column_labels)
df

,UUID,Sample ID,Working compound,Working cell line,Pert Dose,Pert Time,Pert Protocol,Tissue ID,Tissue Passage,Type,Parent,Children,Location,Storage Form,Storage Amount,Pass,Quantity Number of Cells (x1000),Created By
0,6fbd3c55-04bb-4cc9-926d-c25a4a2eb2c1,SKNAS_dbgen1_K975_1,K975_1,SKNAS_dbgen1,,,,,,,6bc9bb34-92bf-41f7-9aa0-220b3403b052,"1736d06b-25a6-40ae-8cc2-e0fe14423242, 6e3e5a5c...",ChESS_dbgen1_plate1,,,,0,
1,32df7236-4639-432a-b2be-0118741c7b3e,SKNAS_dbgen1_T0011026_1,T001-1026_1,SKNAS_dbgen1,,,,,,,6bc9bb34-92bf-41f7-9aa0-220b3403b052,"3b769918-6659-414c-bda4-6796f4adb6df, 2cd58409...",ChESS_dbgen1_plate1,,,,0,
2,ac7003a6-5ec1-443d-9487-d08f19216adc,THP1_dbgen1_JQ1_1,(+)-JQ1_CHESSFEST_1,THP1_dbgen1,,,,,,,90b19820-726d-4086-b74a-199d37899ea5,"74bc0fa6-7ea7-42eb-871e-b2904b67896d, 27528225...",ChESS_dbgen1_plate1,,,,0,


# Jupyter notebook in Scispot

- Scispot --> LabSheets --> Jupyter --> +New --> write script/notebook
- - need jupyterhub server credentials?

- ? can I look up a UUID by sample name?
- - update-by-id to fetch UUID from Sample ID
- ? can I look up UUIDs by manifest name?
- - not yet! 
- ? can I specify other "variable" inputs/params? or can scientist edit the ipynb with those values?
- - A: yes, can code input parameters 
- - ? can I read in other sheets into pandas?
- - - nm don't worry about this for now...


### tips
- update-by-id to fetch UUID from Sample ID 
- 

In [ ]:
## TODO: can I filter for location `ChESS_dbgen1_plate1` ?











In [ ]:
# someday I should actually make objects for scispot stuff...
# TODO: read in headers from API? so that if they change, this isn't all hardcoded...
class scispot_managers:
    def __init__( self, data, ):
        self.sample = pd.DataFrame()
        self.protein = pd.DataFrame()
        self.peptide = pd.DataFrame()
        
    def sample_to_protein( self, input ):
        self.peptide[ 'new_column' ] = input

def initiate_sample_df():
    df = pd.DataFrame(columns=['Sample ID',
                                 'Working compound',
                                 'Working cell line',
                                 'Pert Dose',
                                 'Pert Time',
                                 'Pert Protocol',
                                 'Tissue ID',
                                 'Tissue Passage',
                                 'Type',
                                 'Storage Form',
                                 'Storage Amount',
                                 'Pass',
                                 'Quantity Number of Cells (x1000)',
                                 'Created By'])
    return(df)

def initiate_protein_df(sample_df, frx):
    df = pd.DataFrame(columns=['Protein ID',
                                 'Sample ID',
                                 'Protocol ID',
                                 'Extraction Date',
                                 'Extraction Scientist',
                                 'Extraction Fraction',
                                 'Extraction Notes',
                                 'Protein Concentration (ug/mL)',
                                 'BCA Results',
                                 'Location',
                                 'Storage Form',
                                 'Storage Amount (uL)',
                                 'Pass',
                                 'Created By',
                                 'Creation Date'])
    
    df['Sample ID'] = sample_df['Sample ID']
    df['Protein ID'] = sample_df['Sample ID'].astype(str) + '_' + frx
    df['Extraction Fraction'] = frx
    # TODO add datetime ['Creation Date']
    # TODO add log-in ['Created By']
    # TODO inherit Sample ID UUID from Sample ID 
    # TODO add 
    
    return(df)

def initiate_peptide_df(input_df):
    df = pd.DataFrame(columns=['Digestion ID',
                                 'Protein ID',
                                 'Digestion Date',
                                 'Protocol ID',
                                 'Digestion Scientist',
                                 'Protein Input',
                                 'Reduction Chemical',
                                 'Reduction Dose',
                                 'Alkylation Chemical',
                                 'Alkylation Dose',
                                 'Protease Chemical',
                                 'Protease Dose',
                                 'Quench Chemical',
                                 'Quench Dose',
                                 'Storage Location',
                                 'Storage Form',
                                 'Storage Concentration',
                                 'Pass',
                                 'Location',
                                 'Created By'])
    
    df['Protein ID'] = input_df['Protein ID']
    df['Digestion ID'] = input_df['Protein ID'].astype(str) + '_peptide'
    # add datetime ['Creation Date']
    # add log-in ['Created By']
    
    # populate some default values
    df['Reduction Chemical'] = 'DTT'
    df['Alkylation Chemical'] = 'IAA'
    df['Protease Chemical'] = 'Trypsin'
    
    return(df)

## TODO: what columns to inherit from Sample? Plate position? populate default values?

In [ ]:
# read in a Scispot plate TSV export of Sample Manager
## TODO possible to pull a plate of samples with some manifest name/location from API?
sample_fi = "../data/Sample-2022-11-15T22_47_43.tsv"
input_df = pd.read_table(sample_fi, sep='\t', engine="python")
input_df.head()

## GENERATE PROTEIN SAMPLES
## TODO: this is pretty inelegant, could perhaps reduce this to a list of fractions to generate?

# generate protein samples: cytosol
protein_df_cyto = initiate_protein_df(input_df, 'Cytosolic')

# generate protein samples: nucleoplasm
protein_df_nucleo = initiate_protein_df(input_df, 'Nucleoplasm')

# generate protein samples: euchromatin
protein_df_euch = initiate_protein_df(input_df, 'Euchromatin')

# generate protein samples: heterochromatin
protein_df_het = initiate_protein_df(input_df, 'Heterochromatin')

# generate protein samples: insoluble
protein_df_ins = initiate_protein_df(input_df, 'Insoluble')

# append all protein to one protein dataframe
protein_df = protein_df_cyto.append(protein_df_nucleo.append(protein_df_euch.append(protein_df_het.append(protein_df_ins))))

# 



In [ ]:

print(protein_df.head())
print(peptide_df.head())